# 1. Import Library

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import numpy as np

# 2. Mengunduh dataset shakesphere

In [3]:
!wget --no-check-certificate \
  https://huggingface.co/arnavmahapatra/gpt2-sonnet-generators/blob/main/shakespeare.txt \
  -O sonnets.txt

--2024-11-15 08:29:47--  https://huggingface.co/arnavmahapatra/gpt2-sonnet-generators/blob/main/shakespeare.txt
Resolving huggingface.co (huggingface.co)... 18.238.109.102, 18.238.109.92, 18.238.109.52, ...
Connecting to huggingface.co (huggingface.co)|18.238.109.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1094068 (1.0M) [text/html]
Saving to: ‘sonnets.txt’

sonnets.txt         100%[===================>]   1.04M  4.38MB/s    in 0.2s    

2024-11-15 08:29:47 (4.38 MB/s) - ‘sonnets.txt’ saved [1094068/1094068]



# 3. Mendefinisikan Tokenizer dan Menyiapkan Training Data

Langkah  selanjutnya  yaitu  melakukan  proses  tokenisasi  dan  menyiapkan  data  yang  akan dilatih.  tokenisasi  adalah  proses  untuk  membagi  teks  yang  dapat  berupa  kalimat,  paragraf atau dokumen, menjadi token-token/bagian-bagian tertentu. Pada proses ini suatu kata yang ada pada  data sheet akan disimbolkan dengan angka secara acak sampai jumlah  kata yang ada  pada  datasheet  tersebut.  Pada  datasheet  ini  jumlah  katanya  yaitu  sampai  2900. Contohnya yaitu angka 1 didefinisikan untuk kata “and”, 2 untuk kata “the” dan seterusnya. Berikut adalah kodingan serta output dari proses tokenizer.

In [3]:
tokenizer = Tokenizer()

data = open('sonnets.txt').read()

corpus = data.lower().split('\n')

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

{'and': 1, 'the': 2, 'to': 3, 'of': 4, 'my': 5, 'i': 6, 'in': 7, 'that': 8, 'thy': 9, 'thou': 10, 'with': 11, 'for': 12, 'a': 13, 'but': 14, 'love': 15, 'is': 16, 'thee': 17, 'not': 18, 'me': 19, 'so': 20, 'you': 21, 'all': 22, 'be': 23, 'his': 24, 'when': 25, 'as': 26, 'which': 27, 'your': 28, 'this': 29, 'it': 30, 'doth': 31, 'by': 32, 'or': 33, 'do': 34, 'from': 35, 'then': 36, 'on': 37, 'no': 38, 'are': 39, 'their': 40, 'shall': 41, 'what': 42, 'have': 43, 'mine': 44, 'time': 45, 'if': 46, 'sweet': 47, 'more': 48, 'beauty': 49, 'art': 50, 'than': 51, 'where': 52, 'nor': 53, 'can': 54, 'should': 55, 'will': 56, 'now': 57, 'o': 58, 'how': 59, 'they': 60, 'he': 61, 'yet': 62, 'hath': 63, 'eyes': 64, 'make': 65, 'thine': 66, 'fair': 67, 'eye': 68, 'him': 69, 'one': 70, 'still': 71, 'like': 72, 'being': 73, 'every': 74, 'own': 75, 'praise': 76, 'some': 77, 'true': 78, 'such': 79, 'live': 80, 'heart': 81, 'see': 82, 'though': 83, 'day': 84, 'world': 85, 'were': 86, 'those': 87, 'give': 8

In [4]:
input_sequences = []

for line in corpus:
  token_list = tokenizer.texts_to_sequences ([line])[0]
  #print("LIST"+str(token_list))
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)


##pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])

print(max_sequence_len, total_words)

input_sequences = np.array(pad_sequences(input_sequences, padding='pre', maxlen=max_sequence_len))

#create predictors and labels
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]

ys = tf.keras.utils.to_categorical (labels, num_classes=total_words)

11 2900


# 4. Mendefinisikan Arsitektur Model

Langkah selanjutnya yaitu mendefinisikan arsitektur , pada tahap ini kita menentukan berapa total kata yang ada pada artikel tersebut kemudian menentukan urutan setiap kata yang ada pada  artikel  tersebut.  Berikut  adalah  kodingan  serta  output  dari  mendefinisikan  arsitektur modelnya.

In [15]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(96)))
model.add(Dense(total_words//2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))

print(model.summary())


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_12 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_13 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


# 5. Training Data

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 62s 143ms/step - accuracy: 0.0237 - loss: 7.7045
Epoch 2/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 55s 145ms/step - accuracy: 0.0209 - loss: 6.4439
Epoch 3/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 80s 139ms/step - accuracy: 0.0265 - loss: 6.3765
Epoch 4/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 82s 138ms/step - accuracy: 0.0268 - loss: 6.3082
Epoch 5/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 82s 137ms/step - accuracy: 0.0316 - loss: 6.2169
Epoch 6/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 83s 139ms/step - accuracy: 0.0419 - loss: 6.0480
Epoch 7/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 83s 142ms/step - accuracy: 0.0393 - loss: 5.9812
Epoch 8/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 83s 147ms/step - accuracy: 0.0433 - loss: 5.9233
Epoch 9/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 78s 137ms/step - accuracy: 0.0506 - loss: 5.7860
Epoch 10/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 81s 135ms/step - accuracy: 0.0470 - loss: 5.7064


# 6. Membuat Perintah Untuk 100 Kata Selanjutnya

Langkah selanjutnya yaitu membuat perintah untuk RNN supaya dapat memprediksi 100kata selanjutnya pada artikel yang ingin dibuat berdasarkan datasheet yang sudah diunduh pada langkah sebelumnya. Pada langkah ini RNN akan dipancing untuk memprediksi kataselanjutnya berdasarkan seed textyang sudah ditentukan sebelumnya.

In [19]:
def predict_next_words(seed_text, next_words):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predict_x = model.predict(token_list)
    predicted = np.argmax(predict_x,axis=1)
    #predicted model.predict_classes(token list, verbose-8)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word

  print(seed_text)
  return seed_text

# 7. Output Artikel dengan Seed Text

In [22]:
seed_text = '1THREEPIO'
next_words = 100

generated_text = predict_next_words(seed_text, next_words)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━

In [21]:
seed_text = 'why lovest thou that which not gladly'
generated_text = predict_next_words(seed_text, next_words)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━